# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import qgrid
import yaml

import ipywidgets as widgets
from pathlib import Path

from interface import DupInterface, layouts
import table

# QGrid

In [ ]:
with open(r'jsl\jsl.yaml', 'r') as file:
    cfg = yaml.load(file, Loader=yaml.SafeLoader)

di = DupInterface(
        pd.DataFrame(),
        qgrid_opts=cfg.get('qgrid', {}),
        default_columns=cfg.get('default_columns', None),
        dup_pre_sel=cfg.get('default_duplicates', None)
)
di.loader.children[1].value = r'D:/Picture Cleanup/jsl/df.pkl'
display(di.widget)